# Find the stimuli that maximally differentiates LN model from the Convnet trained on natural scenes

In [28]:
import numpy as np
from os.path import expanduser
import os
import json
import theano
import pyret.filtertools as ft
import pyret.visualizations as pyviz
import deepretina.visualizations as viz
from deepretina.preprocessing import datagen, loadexpt
from keras.models import model_from_json
import h5py
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from scipy.optimize import minimize
from numpy.linalg import norm
%matplotlib inline
#%matplotlib qt

from pylab import rcParams
rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
rcParams['image.interpolation'] = 'nearest'
rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Filenames and data directories

In [4]:
architecture_filename = 'architecture.json'
#whitenoise_data_dir = expanduser('~/Dropbox/deep-retina/saved/lenna.salamander/2015-11-08 15.42.33 convnet/')
#whitenoise_weight_filename = 'epoch018_iter01300_weights.h5' # .63 cc on held-out
naturalscenes_data_dir = expanduser('~/Dropbox/deep-retina/saved/lenna.salamander/2015-11-07 16.52.44 convnet/')
naturalscenes_weight_filename = 'epoch038_iter02700_weights.h5' # .53 cc on held-out
ln_data_dir = expanduser('~/Dropbox/deep-retina/saved/lenna.nirum/2015-11-08 04.41.18 LN/')
ln_weight_filename = 'epoch010_iter00750_weights.h5' # .468 cc on held-out

## Load convnet trained on natural scenes

In [3]:
naturalscenes_architecture_data = open(naturalscenes_data_dir + architecture_filename, 'r')
naturalscenes_architecture_string = naturalscenes_architecture_data.read()
naturalscenes_model = model_from_json(naturalscenes_architecture_string)
naturalscenes_model.load_weights(naturalscenes_data_dir + naturalscenes_weight_filename)

## Load LN model

In [5]:
ln_architecture_data = open(ln_data_dir + architecture_filename, 'r')
ln_architecture_string = ln_architecture_data.read()
ln_model = model_from_json(ln_architecture_string)
ln_model.load_weights(ln_data_dir + ln_weight_filename)

In [47]:
def model_separation(stimulus):
    stimulus = stimulus.reshape((1,40,50,50))
    ln_response = ln_model.predict(stimulus)[0][0]
    convnet_response = naturalscenes_model.predict(stimulus)[0][0]
    return -(convnet_response - ln_response)**2

In [48]:
constraint = {}
def unit_norm_constraint(stimulus):
    return (1 - norm(stimulus.ravel()))**2
constraint['fun'] = unit_norm_constraint
constraint['type'] = 'eq'

In [ ]:
initial_guess = np.random.randn(1*40*50*50)
res = minimize(model_separation, x0=initial_guess)